# PROJECT 15 트랜스포머로 만드는 대화형 챗봇

**트랜스 포머 모델을 기반으로 한 인코더-디코더 구조를 바탕으로 챗봇을 제작해 보도록 하겠습니다.**

## 1. 데이터 로드하기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
train_data = pd.read_csv('ChatbotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


ChatbotData.csv 데이터는 질문과 대답의 쌍으로 이루어진 데이터입니다.

In [3]:
print('train data 개수 :', len(train_data))

train data 개수 : 11823


train data 개수는 총 11,823개 입니다.  

## 2. 데이터 전처리하기

데이터 안에 Null 값이 있는지 확인해 보도록 하겠습니다.

In [4]:
print(train_data.isnull().sum())

Q        0
A        0
label    0
dtype: int64


Null 값은 존재하지 않습니다.

데이터에서 ?, ., ! 와 같은 구두점 앞에 공백을 추가하여 다른 문자들과 구분하겠습니다.

In [5]:
def preprocess_strip(train_data):
    questions = []
    answers = []
    
    for q,a in zip(train_data['Q'], train_data['A']):
        q = re.sub(r"([?.!,])", r" \1 ", q)
        a = re.sub(r"([?.!,])", r" \1 ", a)
        
        q = q.strip()
        a = a.strip()
        
        questions.append(q)
        answers.append(a)
        
    return questions, answers

In [6]:
questions, answers = preprocess_strip(train_data)

In [7]:
print("="*20 + " Question " + "="*20)
for i in range(5):
    print(f"{train_data['Q'][i]} → {questions[i]}")
print()

print("="*20 + " Answer " + "="*20)
for i in range(5):
    print(f"{train_data['A'][i]} → {answers[i]}")

==================== Question ====================
12시 땡! → 12시 땡 !
1지망 학교 떨어졌어 → 1지망 학교 떨어졌어
3박4일 놀러가고 싶다 → 3박4일 놀러가고 싶다
3박4일 정도 놀러가고 싶다 → 3박4일 정도 놀러가고 싶다
PPL 심하네 → PPL 심하네

==================== Answer ====================
하루가 또 가네요. → 하루가 또 가네요 .
위로해 드립니다. → 위로해 드립니다 .
여행은 언제나 좋죠. → 여행은 언제나 좋죠 .
여행은 언제나 좋죠. → 여행은 언제나 좋죠 .
눈살이 찌푸려지죠. → 눈살이 찌푸려지죠 .


구두점 앞에 띄어쓰기가 추가되어 분리된 것을 확인할 수 있습니다.

## 3. 단어 집합 생성하기

서브워드텍스트인코더를 사용하여 질문, 답변 데이터로부터 단어 집합을 생성하도록 하겠습니다.

In [8]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

In [9]:
print(tokenizer.vocab_size)

8178


8,178 개의 단어 집합을 생성하였습니다.

시작은 의미하는 시작 토큰 sos 와 종료를 의미하는 종료 토큰 eos 를 단어 집합에 포함시키도록 하겠습니다. 

In [10]:
# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2

In [11]:
print('시작 토큰 번호 :',START_TOKEN[0])
print('종료 토큰 번호 :',END_TOKEN[0])
print('단어 집합의 크기 :',VOCAB_SIZE)

시작 토큰 번호 : 8178
종료 토큰 번호 : 8179
단어 집합의 크기 : 8180


sos 토큰과 eos 토큰을 추가함으로써 단어 집합의 크기가 8,178 개에서 8,180 개로 늘어났습니다.

## 4. 정수 인코딩과 패딩

tokenizer.encode() 를 사용하여 정수 인코딩을 하도록 하겠습니다.  
패딩의 길이는 40으로 설정하도록 하겠습니다.

In [12]:
# 최대 길이를 40으로 정의
MAX_LENGTH = 40

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)

  # 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs

In [13]:
questions, answers = tokenize_and_filter(questions, answers)

In [14]:
print('question shape :', questions.shape)
print('answer shape :', answers.shape)

question shape : (11823, 40)
answer shape : (11823, 40)


In [15]:
# 0번 샘플을 임의로 출력
print("="*30 + " questions[0] " + "="*30)
print(questions[0])
print()

print("="*30 + " answers[0] " + "="*30)
print(answers[0])

============================== questions[0] ==============================
[8178 7915 4207 3060   41 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]

============================== answers[0] ==============================
[8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


## 5. 인코더와 디코더의 입력, 레이블 만들기

tf.data.Dataset.from_tensor_slices() 를 사용하여 tf.data.Dataset 를 생성합니다.  
tf.data.Dataset 을 생성할 때, 인코더의 입력은 토큰 그대로 하고, 인코더의 출력은 맨 처음 토큰을 제거합니다. 그리고 디코더의 입력은 마지막 토큰을 제거합니다.  
이렇게 설정해야 교사 강요를 사용할 수 있습니다,

In [16]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000


dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,    # 디코더의 입력
        'dec_inputs': answers[:, :-1] # 디코더의 입력, 인코더의 출력
    },
    {
        'outputs': answers[:, 1:]  # 디코더의 출력
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [17]:
print("="*30 + " 원천 데이터 " + "="*30)
print(answers[0])
print()

print("="*23 + " 인코더의 출력 및 디코더의 입력 " + "="*23)
print(answers[:1][:, :-1])
print()

print("="*30 + " 디코더의 출력 " + "="*30)
print(answers[:1][:, 1:])
print()

============================== 원천 데이터 ==============================
[8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]

======================= 인코더의 출력 및 디코더의 입력 =======================
[[8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]

============================== 디코더의 출력 ==============================
[[3844   74 7894    1 8179    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]



## 6. 포지셔널 인코딩 구현하기

트랜스 포머는 단어 입력을 순차적으로 받는 방식이 아니기 때문에 단어의 위치 정보를 알아야할 필요가 있습니다. 트랜스 포머는 단어의 위치 정보를 얻기 위해서 각 단어의 임베딩 벡터에 위치 정보를 더하여 모델의 입력으로 사용하는데, 이를 포지셔널 인코딩이라고 합니다.

In [18]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])

    # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

## 7. 스케일 닷-프로덕트 어텐션 구현하기

트랜스 포머에서는 어텐션 값을 구하는 방법으로 스케일 닷 프로덕트 어텐션을 사용합니다.

In [19]:
def scaled_dot_product_attention(query, key, value, mask):
  # query 크기 : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  # key 크기 : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
  # value 크기 : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
  # padding_mask : (batch_size, 1, 1, key의 문장 길이)

  # Q와 K의 곱. 어텐션 스코어 행렬.
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 스케일링
  # dk의 루트값으로 나눠준다.
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 마스킹. 어텐션 스코어 행렬의 마스킹 할 위치에 매우 작은 음수값을 넣는다.
  # 매우 작은 값이므로 소프트맥스 함수를 지나면 행렬의 해당 위치의 값은 0이 된다.
  if mask is not None:
    logits += (mask * -1e9)

  # 소프트맥스 함수는 마지막 차원인 key의 문장 길이 방향으로 수행된다.
  # attention weight : (batch_size, num_heads, query의 문장 길이, key의 문장 길이)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  output = tf.matmul(attention_weights, value)

  return output, attention_weights

## 8. 멀티 헤드 어텐션 구현하기

스케일 닷-프로덕트 어텐션을 여러 개 만들어 다양한 특징에 대한 어텐션을 볼 수 있게 멀티 헤드 어텐션을 사용합니다.

In [20]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    # d_model을 num_heads로 나눈 값.
    # 논문 기준 : 64
    self.depth = d_model // self.num_heads

    # WQ, WK, WV에 해당하는 밀집층 정의
    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    # WO에 해당하는 밀집층 정의
    self.dense = tf.keras.layers.Dense(units=d_model)

  # num_heads 개수만큼 q, k, v를 split하는 함수
  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # 1. WQ, WK, WV에 해당하는 밀집층 지나기
    # q : (batch_size, query의 문장 길이, d_model)
    # k : (batch_size, key의 문장 길이, d_model)
    # v : (batch_size, value의 문장 길이, d_model)
    # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 2. 헤드 나누기
    # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    # (batch_size, query의 문장 길이, d_model)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    # (batch_size, query의 문장 길이, d_model)
    outputs = self.dense(concat_attention)

    return outputs

## 9. 패딩 마스크 구현하기

패딩은 문장의 길이가 서로 다를 때, 모든 문장의 길이를 동일하게 해주는 패딩 마스크 과정을 수행합니다.

In [21]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, key의 문장 길이)
  return mask[:, tf.newaxis, tf.newaxis, :]

## 10. 룩 어헤드 마스킹 구현하기

트랜스 포머는 전체 문장이 문장 행렬로 들어가기 때문에 위치와 상관없이 모든 단어를 참고해서 다음 단어를 예측합니다. 하지만 우리가 원하는 것은 이전 단어들로부터 다음 담어를 예측하는 것이기 때문에, 다음에 나올 단어를 참고하지 않도록 가리는 기법인 룩 어헤드 마스킹을 사용합니다.

In [22]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x) # 패딩 마스크도 포함
  return tf.maximum(look_ahead_mask, padding_mask)

## 11. 인코더 구현하기

하나의 인코더 층은 셀프 어텐션과 피드 포워드 신경망으로 이루어져있습니다. 셀프 어텐션은 멀티 헤드 어텐션으로 병렬적으로 이루어집니다.  
셀프 어텐션과 피드 포워드 신경망을 가지는 하나의 인코더 층을 구현하는 함수는 다음과 같습니다.

In [23]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 멀티-헤드 어텐션 (첫번째 서브층 / 셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': padding_mask # 패딩 마스크 사용
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 포지션 와이즈 피드 포워드 신경망 (두번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

위에서 구현한 인코더 층을 임베딩 층과 포지셔널 포지셔널 인코딩을 연결하고, 인코더 층을 쌓아 트랜스 포머의 인코더를 만듭니다.

In [24]:
def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 인코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

## 12. 디코더 구현하기

하나의 디코더 층은 셀프 어텐션, 인코더-디코더 어텐션, 피드 포워드 신경망으로 이루어져있습니다.  
셀프 어텐션, 인코더-디코더는 멀티 헤드 어텐션으로 병렬적으로 수행합니다.  
셀프 어텐션, 인코더-디코더 어텐션, 피드 포워드 신경망을 가지는 하나의 디코더 층을 구현하는 함수는 다음과 같습니다.

In [25]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  # 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")

  # 패딩 마스크(두번째 서브층)
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 멀티-헤드 어텐션 (첫번째 서브층 / 마스크드 셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': look_ahead_mask # 룩어헤드 마스크
      })

  # 잔차 연결과 층 정규화
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 멀티-헤드 어텐션 (두번째 서브층 / 디코더-인코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1, 'key': enc_outputs, 'value': enc_outputs, # Q != K = V
          'mask': padding_mask # 패딩 마스크
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 포지션 와이즈 피드 포워드 신경망 (세번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

위에서 구현한 디코더의 층은 임베딩 층과 포지셔널 인코딩을 연결하고, 디코더 층을 쌓아 트랜스 포머의 디코더를 만듭니다.

In [26]:
def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 디코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

## 13. 트랜스포머 구현하기

앞에서 구현한 인코더 층 함수와 디코더 층 함수를 사용하여 트랜스포머 함수를 정의합니다.

In [27]:
def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):

  # 인코더의 입력
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 디코더의 입력
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더의 패딩 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더의 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 디코더의 패딩 마스크(두번째 서브층)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더의 출력은 enc_outputs. 디코더로 전달된다.
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) # 인코더의 입력은 입력 문장과 패딩 마스크

  # 디코더의 출력은 dec_outputs. 출력층으로 전달된다.
  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 다음 단어 예측을 위한 출력층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

## 14. 모델 학습하기

하이퍼파라미터의 값은 아래와 같이 설정합니다.  

${d}_{model} = 256$  

num_layers = 2  

num_heads = 8  

${d}_{ff} = 512$  

In [28]:
tf.keras.backend.clear_session()

# Hyper-parameters
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

(1, 8180, 256)
(1, 8180, 256)


학습률과 옵티마이저를 정의하고 모델을 학습하도록 하겠습니다.

In [29]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [30]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [31]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [32]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
185/185 [==============================] - 14s 75ms/step - loss: 1.4436 - accuracy: 0.0259
Epoch 2/50
185/185 [==============================] - 14s 75ms/step - loss: 1.1739 - accuracy: 0.0495
Epoch 3/50
185/185 [==============================] - 14s 74ms/step - loss: 1.0054 - accuracy: 0.0506
Epoch 4/50
185/185 [==============================] - 14s 75ms/step - loss: 0.9308 - accuracy: 0.0541
Epoch 5/50
185/185 [==============================] - 14s 75ms/step - loss: 0.8724 - accuracy: 0.0575
Epoch 6/50
185/185 [==============================] - 13s 73ms/step - loss: 0.8126 - accuracy: 0.0619
Epoch 7/50
185/185 [==============================] - 13s 71ms/step - loss: 0.7465 - accuracy: 0.0679
Epoch 8/50
185/185 [==============================] - 13s 69ms/step - loss: 0.6723 - accuracy: 0.0758
Epoch 9/50
185/185 [==============================] - 13s 70ms/step - loss: 0.5928 - accuracy: 0.0843
Epoch 10/50
185/185 [==============================] - 13s 69ms/step - loss: 0.510

## 15. 챗봇 테스트하기

In [33]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재(마지막) 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 마지막 시점의 예측 단어를 출력에 연결한다.
    # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

In [34]:
def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [35]:
def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

학습된 트랜스 포머에 임의로 문장을 넣어보겠습니다.

In [42]:
output = predict("배고파")

Input: 배고파
Output: 얼른 맛난 음식 드세요 .


In [43]:
output = predict("심심해")

Input: 심심해
Output: 노래 불러 드릴까요 ?  북치기박치기 헥헥헥헥헥헥헥헥헥헥헥헥헥헥헥헥헥헥헥헥헥헥헥


In [44]:
output = predict("게임하고싶어")

Input: 게임하고싶어
Output: 게임하세요 !


In [45]:
output = predict("공부하기싫어")

Input: 공부하기싫어
Output: 공부하면 더 많은 선택을 할 수 있죠 .


In [46]:
output = predict("아파")

Input: 아파
Output: 아프지 않았으면 좋겠어요 .


# [ 결과 - 루브릭 ]
**1. 한국어 전처리를 통해 학습 데이터셋을 구축하였다.**
- 공백과 특수문자 처리, 토크나이징, 병렬데이터 구축의 과정이 적절히 진행되었습니다. :)  

**2. 트랜스포머 모델을 구현하여 한국어 챗봇 모델 학습을 정상적으로 진행하였다.** 
- 구현한 트랜스포머 모델이 한국어 병렬 데이터 학습 시 안정적으로 수렴하였습니다. :)
  
**3. 한국어 입력문장에 대해 한국어로 답변하는 함수를 구현하였다.**
- 한국어 입력문장에 그럴듯한 한국어로 답변을 리턴하였습니다. :)